In [7]:
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.spatial import distance
from itertools import product

In [2]:
pubs_author_funder = pd.read_csv("..//..//..//data//Data//DerivedData//CleanedRawData//pubs_author_funder.csv")
pubs_author_funder.head()

/home/miaoli/conda/envs/funding/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,author_country,funder_country,discipline,year,rel_citation,citation
0,30507292,['Turkey'],['Turkey'],77,2009,0.485,6
1,30574690,"['United States', 'France']","['United States', 'United States', 'United Sta...",83,2009,0.703,9
2,30574691,['Russia'],['Russia'],83,2009,0.312,4
3,30585095,"['Spain', 'Spain']","['Spain', 'Spain', 'Spain', 'Spain', 'Spain', ...",89,2009,0.137,1
4,30585097,"['Germany', 'Spain']",['Spain'],89,2009,0.410,3


In [4]:
pubs_author_funder['author_country'] = pubs_author_funder['author_country'].apply(literal_eval)
pubs_author_funder['funder_country'] = pubs_author_funder['funder_country'].apply(
    lambda x: literal_eval(x) if pd.notnull(x) else 0)

In [5]:
pubs_author_funder=pubs_author_funder[pubs_author_funder.funder_country!=0]
pubs_author_funder['funding_units'] = pubs_author_funder['funder_country'].apply(lambda x: len(x))

In [16]:
cntry_cntry_dis_flow = []
for index, row in pubs_author_funder.iterrows():
    author_country = row['author_country']
    funding_country=row['funder_country']
    num_author=len(author_country)
    dis=row['discipline']
    credit = 1/(num_author)
    year=row['year']
    for funder,author in product(funding_country,author_country):
        templist = [funder,author,year,dis,credit]
        cntry_cntry_dis_flow.append(templist)

In [18]:
cntry_cntry_dis_df=pd.DataFrame(cntry_cntry_dis_flow, 
                                columns=['source','target','year','discipline','unit'])
cntry_cntry_dis_df.head()

,source,target,year,discipline,unit
0,Turkey,Turkey,2009,77,1.0
1,United States,United States,2009,83,0.5
2,United States,France,2009,83,0.5
3,United States,United States,2009,83,0.5
4,United States,France,2009,83,0.5


In [19]:
cntry_cntry_dis_df = cntry_cntry_dis_df.groupby(['source','target','year','discipline'])['unit'].sum().reset_index()
cntry_cntry_dis_df.head()

,source,target,year,discipline,unit
0,Afghanistan,Afghanistan,2010,66,0.500000
1,Afghanistan,Australia,2018,69,0.333333
2,Afghanistan,Bangladesh,2011,47,0.666667
3,Afghanistan,Bolivia,2016,71,0.125000
4,Afghanistan,Canada,2015,71,0.333333


In [20]:
cntry_cntry_dis_df.to_csv("..//..//..//data//Data//DerivedData//Derived//funding_source_target_dis.csv",
                         index=False)